In [1]:
import math
import pylab
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F


seed = 2021
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)

torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1

In [2]:
from collections import OrderedDict
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(24, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(38912, 256), #166400
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
                               
    def forward(self, x):
        
        H=self.fc(x)
        
        Y_prob=self.fc2(H.reshape(H.size(0),-1))

        return Y_prob

In [3]:
from torch.optim.lr_scheduler import _LRScheduler
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [4]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [5]:
xTrain=np.load("../extracted_features/0/GRP-3/input_features_train_grp3.npy")
yTrain=np.load("../extracted_features/0/GRP-3/labels_train_grp3.npy")

xTest=np.load("../extracted_features/0/GRP-3/input_features_test_grp3.npy")
yTest=np.load("../extracted_features/0/GRP-3/labels_test_grp3.npy")


xVal=np.load("../extracted_features/0/GRP-3/input_features_val_grp3.npy")
yVal=np.load("../extracted_features/0/GRP-3/labels_val_grp3.npy")

train_features=np.load("../extracted_features/bl/0/GRP-3/input_features_train_grp3.npy")
labels_train=np.load("../extracted_features/bl/0/GRP-3/labels_train_grp3.npy")

test_features=np.load("../extracted_features/bl/0/GRP-3/input_features_test_grp3.npy")
labels_test=np.load("../extracted_features/bl/0/GRP-3/labels_test_grp3.npy")

val_features=np.load("../extracted_features/bl/0/GRP-3/input_features_val_grp3.npy")
labels_val=np.load("../extracted_features/bl/0/GRP-3/labels_val_grp3.npy")


In [6]:
# print(yTest)

In [7]:
# print(yVal)

In [8]:
# print(yTrain)

In [9]:
print(train_features.shape,labels_train.shape)
print(test_features.shape,labels_test.shape)
print(val_features.shape,labels_val.shape)

(497, 19, 24) (497,)
(119, 19, 24) (119,)
(110, 19, 24) (110,)


In [10]:
print(xTrain.shape,yTrain.shape)
print(xTest.shape,yTest.shape)
print(xVal.shape,yVal.shape)

(512, 19, 24) (512,)
(122, 19, 24) (122,)
(118, 19, 24) (118,)


In [11]:
print(train_features.shape,labels_train.shape)
print(test_features.shape,labels_test.shape)
print(val_features.shape,labels_val.shape)

(497, 19, 24) (497,)
(119, 19, 24) (119,)
(110, 19, 24) (110,)


In [12]:
train_features=train_features.reshape(len(train_features),1,19,24)
test_features=test_features.reshape(len(test_features),1,19,24)
val_features=val_features.reshape(len(val_features),1,19,24)

xTrain = xTrain.reshape(len(xTrain),1,19,24)
xTest = xTest.reshape(len(xTest),1,19,24)
xVal = xVal.reshape(len(xVal),1,19,24)

In [13]:
# xTrain=np.append(train_features,xTrain,axis=0)
# xTest=np.append(test_features,xTest,axis=0)
# xVal=np.append(val_features,xVal,axis=0)
# yTrain=np.append(labels_train,yTrain,axis=0)
# yTest=np.append(labels_test,yTest,axis=0)
# yVal=np.append(labels_val,yVal,axis=0)

In [14]:
batch_size=20

train_data = Dataset(xTrain, yTrain)
test_data=Dataset(xTest,yTest)
val_data=Dataset(xVal,yVal)

# train_data = Dataset(train_features, labels_train)
# test_data=Dataset(test_features,labels_test)
# val_data=Dataset(val_features,labels_val)

train_data_loader = DataLoader(train_data,worker_init_fn=seed_worker, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(test_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(val_data,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

In [15]:
feature_size = 24
shared_layer_size = 512
LR = 0.0001
epoch = 100
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
model.apply(init_weights)
loss_func = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [16]:
from torchsummary import summary
summary(model.cuda(),(19,24))
# torch.save(model, "modelarchitect_final.pt")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 19, 512]          12,800
              ReLU-2              [-1, 19, 512]               0
            Linear-3             [-1, 19, 1024]         525,312
              ReLU-4             [-1, 19, 1024]               0
            Linear-5             [-1, 19, 2048]       2,099,200
              ReLU-6             [-1, 19, 2048]               0
            Linear-7                  [-1, 256]       9,961,728
       BatchNorm1d-8                  [-1, 256]             512
              ReLU-9                  [-1, 256]               0
           Linear-10                    [-1, 1]             257
          Sigmoid-11                    [-1, 1]               0
Total params: 12,599,809
Trainable params: 12,599,809
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Fo

In [17]:
from sklearn.metrics import accuracy_score
# best_model = "./runs/"+ts+"/best_model_CNN_Cuda1.pth"
best_acc1 = 0
# best_model1 = "./saved_models/best_model_CNN_node3"
modelname=[]
truth=[]
preds=[]
model.train()
for it in range(epoch+1):
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=accuracy_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))
    
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()    
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=accuracy_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        print('Best validation accuracy1 ', best_acc1)

EPOCH  0
Train : Loss: 0.0428, Train acc1 : 0.5703
Val : Loss: 0.0306, Val acc1 : 0.6557
Best validation accuracy1  0.6557377049180327
EPOCH  1
Train : Loss: 0.0418, Train acc1 : 0.5994
Val : Loss: 0.0305, Val acc1 : 0.7049
Best validation accuracy1  0.7049180327868853
EPOCH  2
Train : Loss: 0.0384, Train acc1 : 0.5804
Val : Loss: 0.0338, Val acc1 : 0.5000
EPOCH  3
Train : Loss: 0.0324, Train acc1 : 0.5789
Val : Loss: 0.0305, Val acc1 : 0.7377
Best validation accuracy1  0.7377049180327869
EPOCH  4
Train : Loss: 0.0353, Train acc1 : 0.5804
Val : Loss: 0.0332, Val acc1 : 0.6885
EPOCH  5
Train : Loss: 0.0324, Train acc1 : 0.6104
Val : Loss: 0.0323, Val acc1 : 0.7131
EPOCH  6
Train : Loss: 0.0326, Train acc1 : 0.6025
Val : Loss: 0.0276, Val acc1 : 0.7459
Best validation accuracy1  0.7459016393442623
EPOCH  7
Train : Loss: 0.0315, Train acc1 : 0.6293
Val : Loss: 0.0279, Val acc1 : 0.7459
Best validation accuracy1  0.7459016393442623
EPOCH  8
Train : Loss: 0.0320, Train acc1 : 0.6325
Val : L

EPOCH  88
Train : Loss: 0.0088, Train acc1 : 0.8754
Val : Loss: 0.1139, Val acc1 : 0.5738
EPOCH  89
Train : Loss: 0.0030, Train acc1 : 0.9117
Val : Loss: 0.1169, Val acc1 : 0.5656
EPOCH  90
Train : Loss: 0.0040, Train acc1 : 0.8959
Val : Loss: 0.1347, Val acc1 : 0.5738
EPOCH  91
Train : Loss: 0.0028, Train acc1 : 0.9101
Val : Loss: 0.1741, Val acc1 : 0.5492
EPOCH  92
Train : Loss: 0.0027, Train acc1 : 0.9038
Val : Loss: 0.1332, Val acc1 : 0.5820
EPOCH  93
Train : Loss: 0.0017, Train acc1 : 0.9164
Val : Loss: 0.1496, Val acc1 : 0.5738
EPOCH  94
Train : Loss: 0.0018, Train acc1 : 0.9148
Val : Loss: 0.2651, Val acc1 : 0.5574
EPOCH  95
Train : Loss: 0.0015, Train acc1 : 0.9132
Val : Loss: 0.1315, Val acc1 : 0.5656
EPOCH  96
Train : Loss: 0.0040, Train acc1 : 0.8927
Val : Loss: 0.3125, Val acc1 : 0.5902
EPOCH  97
Train : Loss: 0.0015, Train acc1 : 0.9196
Val : Loss: 0.1178, Val acc1 : 0.5656
EPOCH  98
Train : Loss: 0.0018, Train acc1 : 0.9085
Val : Loss: 0.1464, Val acc1 : 0.6148
EPOCH  99


In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.54 0.45 0.55 0.38 0.54 0.70


In [19]:
# torch.save(best_state, "./saved_model-baseline-free/DNN_best_free_speech_F1_52.pth")

In [20]:
xTest=np.load("../extracted_features/0/win-2/GRP-3/input_features_val_grp3.npy")
yTest=np.load("../extracted_features/0/win-2/GRP-3/labels_val_grp3.npy")

xTest = xTest.reshape(len(xTest),1,19,24)

testdataset = Dataset(xTest,yTest)

test_data_loader = DataLoader(testdataset,worker_init_fn=seed_worker,shuffle=True,batch_size=batch_size)

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.61 0.53 0.65 0.45 0.61 0.77
